In [1]:
#visualization
import matplotlib.pyplot as plt

#math tools
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import resample
from scipy.signal import decimate
#machine learning
import torch
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(device)

#utils
from time import time
from os.path import join
from os import listdir


cuda


In [2]:
def timeSince(since):
    now = time()
    s = now - since
    m = np.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [3]:
data_path=join("..","NewHandPD")
folder_path=listdir(data_path)
folder_path.sort()
print(folder_path)

[' Patient Signal(2).zip', 'Healthy Signal(2)', 'HealthyMeander', 'HealthySpiral.zip', 'HelthyCircle.zip', 'PatientCircle.zip', 'PatientMeander.zip', 'PatientSpiral.zip', 'description.html', 'description_fichiers']


In [32]:
def yield_dataset(folder_path):
    for folder in folder_path:
        dataset=[]
        task_path=listdir(join(data_path,folder))
        task_path.sort()
        for task in task_path:
            measures=[]
            path=join(data_path,folder,task)
            
            """with open(path) as file:
                for line in file.readlines():
                    measures.append(line.split(","))"""
            dataset.append(np.loadtxt(path,delimiter=","))#measures
        yield np.concatenate((dataset[0],dataset[1]),axis=0)

In [33]:
raw=next(yield_dataset(folder_path))

print(len(raw))

905


In [ ]:
class Model(torch.nn.Module):
    """
    Args:
        #Vanilla LSTM
        input_size: The number of expected features in the input `x`
        hidden_size: The number of features in the hidden state `h`
        num_layers: Number of recurrent layers. E.g., setting ``num_layers=2``
            would mean stacking two LSTMs together to form a `stacked LSTM`,
            with the second LSTM taking in outputs of the first LSTM and
            computing the final results. Default: 1
        bias: If ``False``, then the layer does not use bias weights `b_ih` and `b_hh`.
            Default: ``True``
        batch_first: If ``True``, then the input and output tensors are provided
            as (batch, seq, feature). Default: ``False``
        dropout: If non-zero, introduces a `Dropout` layer on the outputs of each
            LSTM layer except the last layer, with dropout probability equal to
            :attr:`dropout`. Default: 0
        bidirectional: If ``True``, becomes a bidirectional LSTM. Default: ``False``
        
        #our model
        batch_size : default : 1
        output_size : default : 1
    """
    def __init__(self, input_size, hidden_size,num_layers=1, bias=True,batch_first=False,
                 dropout=0,bidirectional=False, batch_size=1, output_size=1):
        super(Model, self).__init__()
        #Vanilla LSTM
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bias=bias
        self.batch_first=batch_first
        self.dropout=dropout
        self.bidirectional=bidirectional
        #our model
        self.batch_size = batch_size
        self.output_size = output_size

        # Define the LSTM layer
        self.encoder = torch.nn.LSTM(self.input_size, self.hidden_size, self.num_layers,self.bias,self.batch_first,
                            self.dropout,self.bidirectional)
        
        #define the dropout layer
        self.dropout_layer=torch.nn.Dropout(self.dropout)

        # Define the decoder layer
        self.decoder = torch.nn.LSTM(self.hidden_size, self.output_size)
        self.sigmoid = torch.nn.Sigmoid()
                
    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of encoder_out: (seq_len, batch, num_directions * hidden_size)
        # shape of self.hidden: (h_n, c_n), where hidden state h_n and cell state c_n both 
        # have shape (num_layers * num_directions, batch, hidden_size).
        encoder_out, self.hidden = self.encoder(input)
        #print(encoder_out.shape)
        
        if self.bidirectional:
            #sums the outputs : direction left-right and direction right-left
            # encoder_out shape should now be (seq_len, batch,hidden_size)
            encoder_out = encoder_out[: ,: ,: self.hidden_size] + encoder_out[: , :, self.hidden_size: ]
        
        # Only take the output from the final timestep
        drop=self.dropout_layer(encoder_out[-1])
        y_pred = self.linear(drop)
        y_pred = self.sigmoid(y_pred)
        return y_pred.squeeze(0)
    
    def init_hidden(self):
        """h_0 of shape (num_layers * num_directions, batch, hidden_size): 
        tensor containing the initial hidden state for each element in the batch. 
        If the RNN is bidirectional, num_directions should be 2, else it should be 1.

        c_0 of shape (num_layers * num_directions, batch, hidden_size): 
        tensor containing the initial cell state for each element in the batch.
        
        The hidden state is modified in place.
        """
        num_directions=1
        if self.bidirectional:
            num_directions=2
            
        
        self.hidden=torch.zeros(self.num_layers*num_directions, self.batch_size, self.hidden_size),
        torch.zeros(self.num_layers*num_directions, self.batch_size, self.hidden_size)
    
    def init_forget_bias(self):
        """Following advices of Jozefowicz et al. 2015,
        we initialize the bias of the forget gate to a large value such as 1
        In PyTorch, the forget gate bias is stored as b_hf in bias_hh_l[k] : 
        the learnable hidden-hidden bias of the kth layer (b_hi|b_hf|b_hg|b_ho), of shape (4*hidden_size).
        So b_hf == bias_hh_lk[hidden_size:2*hidden_size]
        
        The weights are modified in-place, like init_hidden(self).
        """
        gen=self.modules()
        _=next(gen)#model summary : don't care about it
        lstm=next(gen)
        if not isinstance(lstm,torch.nn.LSTM):
            raise NotImplementedError("the encoder should be an LSTM and should be the first module of the model")
        
        with torch.no_grad():#so the optimizer doesn't know about this ;)
            lstm.bias_hh_l0[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)
            if lstm.bidirectional:
                lstm.bias_hh_l0_reverse[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)
            if lstm.num_layers > 1:
                lstm.bias_hh_l1[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)
                if lstm.bidirectional:
                    lstm.bias_hh_l1_reverse[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)
            if lstm.num_layers > 2:
                lstm.bias_hh_l2[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)    
                if lstm.bidirectional:
                    lstm.bias_hh_l2_reverse[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)
            if lstm.num_layers > 3:
                lstm.bias_hh_l3[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)    
                if lstm.bidirectional:
                    lstm.bias_hh_l3_reverse[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)
            if lstm.num_layers > 4:
                lstm.bias_hh_l4[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)    
                if lstm.bidirectional:
                    lstm.bias_hh_l4_reverse[hidden_size:2*hidden_size]=torch.ones(lstm.hidden_size)
                                    
            if lstm.num_layers>5:
                raise NotImplementedError("you can only have max 5 layers for now")
                
    def count_params(self):
        """returns (total n° of parameters, n° of trainable parameters)"""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        return total_params, trainable_params